In [6]:
import requests
import mysql.connector
from mysql.connector import Error
import schedule
import time
 


In [7]:
# Airtable configuration
AIRTABLE_BASE_ID = 'appGnf8ldkBqyxsdX'
AIRTABLE_API_KEY = 'patb92KmkkX08CuWq.3e77882b6467bc2705d5f93a6468e7d74f8276494194126fe99328891d59f46b'
AIRTABLE_URL = f'https://api.airtable.com/v0/{AIRTABLE_BASE_ID}/'

# MySQL configuration
MYSQL_HOST = 'ugandarugbyunion.c5s2s2ki82x4.eu-north-1.rds.amazonaws.com'
MYSQL_DATABASE = 'ugandarugbyunion'
MYSQL_USER = 'admin'
MYSQL_PASSWORD = '2210kpeteR$$'

In [8]:
# Function to fetch records from Airtable
table_name = "postmatchdata"
def fetch_airtable_records(table_name):
    headers = {
        'Authorization': f'Bearer {AIRTABLE_API_KEY}',
        'Content-Type': 'application/json'
    }
    response = requests.get(f'{AIRTABLE_URL}{table_name}', headers=headers)
    
    if response.status_code == 200:
        return response.json().get('records', [])
    else:
        print(f"Failed to fetch records from Airtable: {response.status_code}, Response: {response.text}")
        return []
records = fetch_airtable_records(table_name)
print(f"✅ {table_name}: {len(records)} records fetched.")

✅ postmatchdata: 100 records fetched.


In [9]:
def upsert_record(cursor, table, record):
    fields = record.get('fields', {})

    if table == 'postmatchdata':
        sql = """
        INSERT INTO postmatchdata (DateColumn, Club, PlayerName, Tries, Conversions, ConversionsMissed, Penalties, PenaltiesMissed, DropGoals, DropGoalsMissed, PenaltyTry, TotalPoints, YellowCards, RedCards, SeriousInjuriesSustained)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            Tries = VALUES(Tries),
            Conversions = VALUES(Conversions),
            ConversionsMissed = VALUES(ConversionsMissed),
            Penalties = VALUES(Penalties),
            PenaltiesMissed = VALUES(PenaltiesMissed),
            DropGoals = VALUES(DropGoals),
            DropGoalsMissed = VALUES(DropGoalsMissed),
            PenaltyTry = VALUES(PenaltyTry),
            TotalPoints = VALUES(TotalPoints),
            YellowCards = VALUES(YellowCards),
            RedCards = VALUES(RedCards),
            SeriousInjuriesSustained = VALUES(SeriousInjuriesSustained);
        """

        # Retrieve each field and handle blanks
        date_column = fields.get('DateColumn')
        club = fields.get('Club')
        player_name = fields.get('PlayerName')
        tries = fields.get('Tries')
        conversions = fields.get('Conversions')
        conversions_missed = fields.get('ConversionsMissed')
        penalties = fields.get('Penalties')
        penalties_missed = fields.get('PenaltiesMissed')
        drop_goals = fields.get('DropGoals')
        drop_goals_missed = fields.get('DropGoalsMissed')
        penalty_try = fields.get('PenaltyTry')
        total_points = fields.get('TotalPoints')
        yellow_cards = fields.get('YellowCards')
        red_cards = fields.get('RedCards')
        serious_injuries_sustained = fields.get('SeriousInjuriesSustained')

        # If the field is None or an empty string, decide how you want it to appear in MySQL
        # Adjust fields as needed; here I'm demonstrating keeping fields blank or defaulting to 0 when necessary
        tries = tries if tries not in (None, '') else 0  # Default to 0 for integers
        conversions = conversions if conversions not in (None, '') else 0
        conversions_missed = conversions_missed if conversions_missed not in (None, '') else 0
        penalties = penalties if penalties not in (None, '') else 0
        penalties_missed = penalties_missed if penalties_missed not in (None, '') else 0
        drop_goals = drop_goals if drop_goals not in (None, '') else 0
        drop_goals_missed = drop_goals_missed if drop_goals_missed not in (None, '') else 0
        penalty_try = penalty_try if penalty_try not in (None, '') else 0
        total_points = total_points if total_points not in (None, '') else 0
        yellow_cards = yellow_cards if yellow_cards not in (None, '') else 0
        red_cards = red_cards if red_cards not in (None, '') else 0
        serious_injuries_sustained = serious_injuries_sustained if serious_injuries_sustained not in (None, '') else 'No Injuries'  # Default message


        values = (
            date_column,
            club,
            player_name,
            tries,
            conversions,
            conversions_missed,
            penalties,
            penalties_missed,
            drop_goals,
            drop_goals_missed,
            penalty_try,
            total_points,
            yellow_cards,
            red_cards,
            serious_injuries_sustained
        )

        print(f"Executing SQL: {sql} with values: {values}")  # Logging

    else:
        print(f"⚠️ Table '{table}' not handled.")
        return

    try:
        cursor.execute(sql, values)
    except Error as err:
        print(f"Error executing SQL for table {table}: {err}")


In [ ]:
# Main function to synchronize Airtable with MySQL
def sync_airtable_to_mysql():
    try:
        connection = mysql.connector.connect(
            host=MYSQL_HOST,
            database=MYSQL_DATABASE,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD
        )
        
        if connection.is_connected():
            cursor = connection.cursor()

            # Fetch current records from MySQL for the players table
            cursor.execute("SELECT DateColumn, Club, PlayerName FROM postmatchdata;")
            existing_records = { (row[0], row[1], row[2]) for row in cursor.fetchall() }  # Set of tuples

            # Fetch records from Airtable specifically for players
            records = fetch_airtable_records('postmatchdata')  # Specify table name
            
            # Prepare to track identifiers fetched from Airtable
            fetched_identifiers = set()

            for record in records:
                identifier = (record['fields'].get('DateColumn'), record['fields'].get('Club'), record['fields'].get('PlayerName'))  # Create tuple for identifiers

                # Upsert record into MySQL
                upsert_record(cursor, 'postmatchdata', record)

                # Track fetched identifiers (ensure they are tuples)
                fetched_identifiers.add(identifier)

            # Deletions: Identify records in MySQL not found in Airtable
            for identifier in existing_records:
                if identifier not in fetched_identifiers:
                    # Make sure to unpack the identifier tuple correctly,
                    # assuming it is structured as (Club, PlayerName, DOB)
                    delete_sql = "DELETE FROM postmatchdata WHERE DateColumn = %s AND Club = %s AND PlayerName = %s;"
                    cursor.execute(delete_sql, identifier)  


            connection.commit()  # Commit all changes
            print("Data synchronized successfully, including deletions.")

    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

# Schedule the sync function to run every minute
schedule.every(1).minutes.do(sync_airtable_to_mysql)

# Keep the script running to maintain the scheduling
while True:
    schedule.run_pending()
    time.sleep(1)  # Sleep to prevent high CPU usage

Executing SQL: 
        INSERT INTO postmatchdata (DateColumn, Club, PlayerName, Tries, Conversions, ConversionsMissed, Penalties, PenaltiesMissed, DropGoals, DropGoalsMissed, PenaltyTry, TotalPoints, YellowCards, RedCards, SeriousInjuriesSustained)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            Tries = VALUES(Tries),
            Conversions = VALUES(Conversions),
            ConversionsMissed = VALUES(ConversionsMissed),
            Penalties = VALUES(Penalties),
            PenaltiesMissed = VALUES(PenaltiesMissed),
            DropGoals = VALUES(DropGoals),
            DropGoalsMissed = VALUES(DropGoalsMissed),
            PenaltyTry = VALUES(PenaltyTry),
            TotalPoints = VALUES(TotalPoints),
            YellowCards = VALUES(YellowCards),
            RedCards = VALUES(RedCards),
            SeriousInjuriesSustained = VALUES(SeriousInjuriesSustained);
         with values: ('2025-02-22', 'Entebbe Monger